## tradeSeq for DE along Müllerian rostro-caudal axis in scRNA-seq mesenchymal cells (axis imputed from ISS)

In [ ]:
#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("tradeSeq")

In [ ]:
#install.packages('igraph')

In [ ]:
ip = as.data.frame(installed.packages()[,c(1,3:4)])
ip = ip[is.na(ip$Priority),1:2,drop=FALSE]
head(ip)

In [ ]:
library(tradeSeq)
library(SingleCellExperiment)
library(slingshot)

In [ ]:
adata <- readRDS("/lustre/scratch126/cellgen/team292/vl6/mese_mullerian_withpseudospace.rds")
adata


In [ ]:
counts <- assay(adata, "X")

In [ ]:
set.seed(7)
pseudotime <- colData(adata)$FemaleReproductiveAxis
cellWeights <- rep(1, 17430)

In [ ]:
#icMat <- evaluateK(counts = counts, sds = SlingshotDataSet(adata), k = 3:10, 
#                   nGenes = 200, verbose = T)

In [ ]:
#BiocManager::install("scran")

In [ ]:
library(scran)

In [ ]:
libsizes <- colSums(counts)
size.factors <- libsizes/mean(libsizes)
logcounts(adata) <- log2(t(t(counts)/size.factors) + 1)
assayNames(adata)

In [ ]:
# Filter genes with non-zero biological components
getVariableGenes <- function(adata,fdr_thresh=NULL,var_thresh=0) {
  gene_var <- modelGeneVar(adata)
  genes <- getTopHVGs(gene_var, var.threshold =var_thresh ,fdr.threshold = fdr_thresh)
  return(genes)
}

In [ ]:
genes <- getVariableGenes(adata)

In [ ]:
length(genes)

In [ ]:
dim(counts[genes,])

In [ ]:
sce <- fitGAM(counts = counts[genes,], pseudotime = pseudotime, cellWeights = cellWeights, nknots = 10,
                verbose = TRUE, parallel = TRUE)

In [ ]:
table(rowData(sce)$tradeSeq$converged)

In [ ]:
adata <- adata[genes, ]

In [ ]:
adata

### ASSOCIATION TEST

A first exploration of the data analysis may consist of checking whether gene expression is associated with a particular lineage. The statistical test performed here, implemented in the associationTest function, is testing the null hypothesis that all smoother coefficients are equal to each other. This can be interpreted as testing whether the average gene expression is significantly changing along pseudotime.



In [ ]:
assoRes <- associationTest(sce)
head(assoRes)

In [ ]:
write.csv(assoRes, file = '/nfs/team292/vl6/FetalReproductiveTract/manuscript_v1/supp_tables/SupplementaryTable8_müllerian_mese_scrnaseq.csv')


In [ ]:
dim(assoRes)

In [ ]:
assoRes['HOXC8',]

In [ ]:
assoRes = assoRes[assoRes$pvalue < 0.001,]
dim(assoRes)

In [ ]:
assoRes = assoRes[assoRes$waldStat > 500,]
dim(assoRes)

In [ ]:
assoRes['WNT4',]

In [ ]:
assoRes = assoRes[order(assoRes$waldStat, decreasing = TRUE),]

In [ ]:
# Get the index of the DataFrame
tops <- row.names(assoRes)[1:50]

In [ ]:
tops

In [ ]:
library(ggplot2)

In [ ]:
for (gene in tops) {
    print(gene)
    print(plotSmoothers(sce, counts, gene = gene, alpha = 0) + ggtitle(gene) +
                          theme(title = element_text(size = 20, face = 'bold'),
                                axis.title = element_text(size = 20),
                                axis.text = element_text(size = 20),
                                legend.text = element_text(size = 20),
                                legend.title = element_text(size = 20, face = "bold")))
}

In [ ]:
sce

In [ ]:
model_vals_df <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100) # output is 708 genes x 100 pseudospace values 

In [ ]:
model_vals_mtx <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100, tidy=FALSE)

In [ ]:
dim(model_vals_mtx) # 1207 genes x 100 pseudospace values 

In [ ]:
model_vals_mtx[1:10, 1:20]

In [ ]:
write.csv(model_vals_mtx, file = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/mese_femalereproductiveaxis_scrnaseq_downsampled_fitted_values_tradeseq.csv')


In [ ]:
hoxa10 <- plotSmoothers(sce, counts, gene = 'HOXA10', alpha =0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA10") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.95)
hoxa10

In [ ]:
hoxa9 <- plotSmoothers(sce, counts, gene = 'HOXA9', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA9") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.6)
hoxa9

In [ ]:
hoxa11 <- plotSmoothers(sce, counts, gene = 'HOXA11', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA11") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.95)
hoxa11

In [ ]:
hoxa13 <- plotSmoothers(sce, counts, gene = 'HOXA13', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA13") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.4)
hoxa13

In [ ]:
hoxc6 <- plotSmoothers(sce, counts, gene = 'HOXC6', alpha =0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)") + ggtitle("HOXC6") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.57)
hoxc6

In [ ]:
hoxc5 <- plotSmoothers(sce, counts, gene = 'HOXC5', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXC5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold")) + theme_light()+ ylim(0, 0.5)
hoxc5

In [ ]:
HOXC4 <- plotSmoothers(sce, counts, gene = 'HOXC4', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXC4") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.65)
HOXC4

In [ ]:
hoxa7 <- plotSmoothers(sce, counts, gene = 'HOXA7', alpha =0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA7") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1)
hoxa7

In [ ]:
hoxa5 <- plotSmoothers(sce, counts, gene = 'HOXA5', alpha =0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.75)
hoxa5

In [ ]:
library(gridExtra)


In [ ]:
p <- grid.arrange(hoxa5, hoxa9, hoxc5, hoxa10, hoxc6,  hoxa11, hoxa7,     hoxa13, 
                  nrow = 4, ncol = 2)

In [ ]:
ggsave('/home/jovyan/RepTract/scrnaseq_femalereproductiveaxis_hox_mese_tradeseq.pdf',
       plot = p,
       width = 8, height = 13, dpi = 300)

In [ ]:
lgr5 <- plotSmoothers(sce, counts, gene = 'LGR5', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("LGR5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 2)
lgr5

In [ ]:
tspan8 <- plotSmoothers(sce, counts, gene = 'TSPAN8', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("TSPAN8") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.7)
tspan8

In [ ]:
cd36 <- plotSmoothers(sce, counts, gene = 'CD36', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("CD36") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 2)
cd36

In [ ]:
p2 <- grid.arrange(lgr5, tspan8, cd36,
                  nrow = 3, ncol = 1)

In [ ]:
ggsave('/home/jovyan/RepTract/scrnaseq_femalereproductiveaxis_fallopian_mese_tradeseq.pdf',
       plot = p2,
       width = 3.75, height = 9, dpi = 300)